In [46]:
# Libraries to access external data
# PyVO provides generic access to VO protocols (ex: TAP)
import pyvo
# astroquery modules are specific to astronomy services, there are not necessarily VO,
# but more and more do switch to using VO protocols behind the scenes these days
# here we import the MOCServer and Vizier modules
from astroquery.mocserver import MOCServer

# The mocpy library is an astropy-affiliated library
# It allows to manipulate MOCs
from mocpy import STMOC, MOC, TimeMOC, WCS

# General astropy utilitary methods and classes
from astropy.coordinates import Longitude, Latitude, Angle, SkyCoord
from astropy.time import Time, TimeDelta
import astropy.units as u

# For vizaulisation
import matplotlib.pyplot as plt
from ipyaladin import Aladin


# Accessing data through space and time criteria

## This notebook focuses on the use of the Multi-Order Coverage (MOC) and Space-Time MOC (STMOC) for querying and analyzing astronomical data. 

### Key words:

**Hierarchical Progressive Survey (HiPS)**: HiPS describes a generic method for packaging, storing, querying, and describing astronomical data. It stores data in a folder structure that mimics the [HEALPix tesselation of the sky](https://healpix.jpl.nasa.gov/pdf/intro.pdf).

**Multi-Order Coverage (MOC)**: MOC is a data structure used to describe arbitrary regions of the sky with variable resolution. It is particularly useful for handling large-scale astronomical surveys and for efficiently querying regions of interest.

**Space-Time MOC (STMOC)**: STMOC extends the concept of MOC to include the temporal dimension, allowing for the representation of regions of the sky that change over time. This is useful for time-domain astronomy, where the visibility of sources varies.

### Scientific Motivation:

Gamma-Ray Bursts (GRBs) are among the most energetic and mysterious phenomena in the universe. These brief but intense flashes of gamma rays are believed to originate from cataclysmic events such as the collapse of massive stars or the merger of neutron stars. Observing GRBs across multiple wavelengths and with different telescopes provides crucial insights into their nature and origins. In this study, we focus on GRBs observed by the Fermi Gamma-ray Space Telescope and the XMM-Newton X-ray Observatory, and demonstrate how the use of Space-Time Multi-Order Coverage (STMOC) can help identify and analyze these fascinating events.

## The HEALPix grid and its link to HiPS surveys

HiPS is a way of storing data sets (it can be catalogs, or images for example) that follows the HEALPix tesselation of the sky. This is particularly optimized for vizualisation.

![HiPS_and_the_HEALPix_grid](HiPS_and_the_HEALPix_grid.png)

Figure 1: Standards to represent the sky, or parts of the sky. (a) The HEALPix grid (b) The HiPS standard (c) Visualisation in ipyaladin (d), and a MOC.

### The HEALPix grid

With the HEALPix grid, the sphere is mapped by equal-area tiles. In **Figure 1a**, we show an orthographic representation of the 12 tiles at order zero and of the 768 tiles at order 3. This is extracted from [Gorski *et.al.*](https://iopscience.iop.org/article/10.1086/427976/pdf).

`mocpy` is a python library that allows to manipulate MOCs. MOCs are made of HEALPix cells (see **Figure 1d** and the next parts where we'll look at MOCs in more details), so mocpy has a few methods that allow to get information about the HEALPix tesselation.

In [2]:
# we can check the number of cells at order 0 in the HEALPix tesselation
MOC.n_cells(0)

12

In [3]:
# or at an other order
MOC.n_cells(3)

768

In [45]:
# we could also have calculated the order 3 by knowing that
# each cell is subdivised in four parts at the next order
n_cells_order3 = MOC.n_cells(0) * 4**3
n_cells_order3

768

Let's vizualize the HEALPix grid in the ipyaladin widget, see **Figure 1c** to see where to find the parameter. If you zoom in and out and move like crazy (or if you have a slower internet connexion), you'll see the images that appear as they are downloaded by ipyaladin. They have the shape of HEALPix cells. The more you zoom in, the more precise (meaning higher resolution) the printed image will be.

In [5]:
aladin = Aladin()
aladin

Aladin(init_options=['_fov', '_target', 'background_color', 'coo_frame', 'full_screen', 'grid_color', 'grid_op…

We can duplicate this view by right clicking on the cell and selecting `Create new view for cell output`. The view can then be dragged anywhere you prefer. This allows to always have an eye on the widget while exploring the data.
In the `Output View`, it is also quite nice to use the `fullscreen` button, the one in the top right corner of the widget.

If you zoom in **a lot**, you will see that the pixels correspond to the tiles of the HEALPix grid at order 18 for the DSS survey that is displayed by default in `ipyaladin`. You can find out which resolution it corresponds to with an other of `mocpy`'s methods:

In [6]:
MOC.order_to_spatial_resolution(18)

<Angle 3.90368159e-06 rad>

Which we can also convert to arcseconds (or any other angle unit, try it out!)

In [7]:
MOC.order_to_spatial_resolution(18).to("arcsec")

<Angle 0.80519213 arcsec>

### The HiPS standard

The images displayed by Aladin follow the HiPS standard, in which a folder corresponds to a HEALPix order, see figure1b. In figure 1b, the example images are from a SDO/AIA observation of the Sun. At all orders, the images have the same number of pixels (here 512*512), but they represent a different sky fraction. We see that the pixel 0 at order 3 contains the pixel 0 and 1 of the more precise order.

### Finding HiPS surveys

#### Fermi

More and more astronomical data centers start to offer HiPS surveys for sky vizualizations.

One way of finding HiPS surveys is to use `ipyaladin`'s interface, see Figure2, and follow the steps to get the two MeerKAT HiPS.

![findind_HiPS](finding_hips_with_ipyaladin.png)
Figure 2: Finding a HiPS with `ipyaladin`'s interface.

Nothing appears? Let's move the target to the galactic center and zoom in a bit:

In [8]:
aladin.target = "Galactic Center"
aladin.fov = 3 # this should be expressed in degrees

The `Browse HiPS` button calls one of the less known services of the CDS, the MOCServer. The search bar only works if you already know more or less the name of the survey. For more control on search parameters, we will use the module `mocserver` from the `astroquery` library.

Here we query the MOC server to find HiPS datasets related to Fermi observations. Fermi cameras are sensitive to sources that emit in gamma-rays. GRBs are the ideal candidate for Fermi.

In [9]:
fermi_hips = MOCServer.find_datasets(meta_data="obs_description=*Fermi*&&hips_*=*", casesensitive=False)

In [10]:
len(fermi_hips)

9

We found nine HiPS surveys with the word `Fermi` in their descriptions. The object that we got from astroquery is a table containing information about the HiPS surveys. Let's inspect its column names to see what it contains.

In [11]:
fermi_hips.colnames

['hips_service_url_2',
 'hips_tile_width',
 'obs_copyright',
 'obs_description',
 'dataproduct_type',
 'bib_reference_url',
 'dataproduct_subtype',
 'hips_rgb_red',
 'hipsgen_params_2',
 'hipsgen_params_3',
 'hips_version',
 'hips_initial_dec',
 'hipsgen_date_3',
 'hipsgen_date_2',
 'obs_copyright_url',
 'hips_rgb_green',
 'hips_status_2',
 'hips_status_1',
 'em_min',
 'hips_order_min',
 'moc_access_url',
 'hips_service_url_3',
 'obs_ack',
 'moc_order',
 'client_application',
 'em_max',
 'hips_frame',
 'hipsgen_date_1',
 'hips_creation_date',
 'client_category',
 'hips_status_3',
 'hipsgen_date_4',
 'obs_collection',
 'hips_release_date',
 'hips_creator',
 'hips_pixel_scale',
 'hips_builder',
 'prov_progenitor',
 'hips_initial_ra',
 'obs_initial_ra',
 'obs_initial_fov',
 'hips_service_url',
 'bib_reference',
 'ID',
 'hips_pixel_bitpix',
 'hips_pixel_cut',
 'hips_estsize',
 'hips_doi',
 'moc_time_range',
 't_min',
 'obs_regime',
 'hips_status',
 'hipsgen_date',
 'hipsgen_params',
 'moc_

We can check the descriptions of the surveys, for example

In [12]:
fermi_hips[["obs_description", "ID"]]

obs_description,ID
object,str20
"Launched on June 11, 2008, the Fermi Gamma-ray Space Telescope observes the cosmos using the highest-energy form of light. This survey sums all data observed by the Fermi mission up to week 396. This version of the Fermi survey are intensity maps where the summed counts maps are divided by the exposure for each pixel. We anticipate using the HEASARC's Hera capabilities to update this survey on a roughly quarterly basis. Data is broken into 5 energy bands : 30-100 MeV Band 1, 100-300 MeV Band 2, 300-1000 MeV Band 3, 1-3 GeV Band 4 , 3-300 GeV Band 5. The SkyView data are based upon a Cartesian projection of the counts divided by the exposure maps. In the Cartesian projection pixels near the pole have a much smaller area than pixels on the equator, so these pixels have smaller integrated flux. When creating large scale images in other projections users may wish to make sure to compensate for this effect the flux conserving clip-resampling option.",CDS/P/Fermi/3
"Launched on June 11, 2008, the Fermi Gamma-ray Space Telescope observes the cosmos using the highest-energy form of light. This survey sums all data observed by the Fermi mission up to week 396. This version of the Fermi survey are intensity maps where the summed counts maps are divided by the exposure for each pixel. We anticipate using the HEASARC's Hera capabilities to update this survey on a roughly quarterly basis. Data is broken into 5 energy bands : 30-100 MeV Band 1, 100-300 MeV Band 2, 300-1000 MeV Band 3, 1-3 GeV Band 4 , 3-300 GeV Band 5. The SkyView data are based upon a Cartesian projection of the counts divided by the exposure maps. In the Cartesian projection pixels near the pole have a much smaller area than pixels on the equator, so these pixels have smaller integrated flux. When creating large scale images in other projections users may wish to make sure to compensate for this effect the flux conserving clip-resampling option.",CDS/P/Fermi/4
"Launched on June 11, 2008, the Fermi Gamma-ray Space Telescope observes the cosmos using the highest-energy form of light. This survey sums all data observed by the Fermi mission up to week 396. This version of the Fermi survey are intensity maps where the summed counts maps are divided by the exposure for each pixel. We anticipate using the HEASARC's Hera capabilities to update this survey on a roughly quarterly basis. Data is broken into 5 energy bands : 30-100 MeV Band 1, 100-300 MeV Band 2, 300-1000 MeV Band 3, 1-3 GeV Band 4 , 3-300 GeV Band 5. The SkyView data are based upon a Cartesian projection of the counts divided by the exposure maps. In the Cartesian projection pixels near the pole have a much smaller area than pixels on the equator, so these pixels have smaller integrated flux. When creating large scale images in other projections users may wish to make sure to compensate for this effect the flux conserving clip-resampling option.",CDS/P/Fermi/5
"Launched on June 11, 2008, the Fermi Gamma-ray Space Telescope observes the cosmos using the highest-energy form of light. This survey sums all data observed by the Fermi mission up to week 396. This version of the Fermi survey are intensity maps where the summed counts maps are divided by the exposure for each pixel. We anticipate using the HEASARC's Hera capabilities to update this survey on a roughly quarterly basis. Data is broken into 5 energy bands : 30-100 MeV Band 1, 100-300 MeV Band 2, 300-1000 MeV Band 3, 1-3 GeV Band 4 , 3-300 GeV Band 5. The SkyView data are based upon a Cartesian projection of the counts divided by the exposure maps. In the Cartesian projection pixels near the pole have a much smaller area than pixels on the equator, so these pixels have smaller integrated flux. When creating large scale images in other projections users may wish to make sure to compensate for this effect the flux conserving clip-resampling option.",CDS/P/Fermi/color
"Fermi-LAT all-sky survey in Band 1 (30-100 MeV), at an angular 

#### XMM-Newton

GRBs can be studied in conjunction with other instruments spanning wide range of wavelengths in order to understand the various properties of the GRB. The bursts of gamma-rays produced by the GRBs collide with each other and also the interstellar medium and produce what we call the "after glow". The "after glow" contains electromagnetic radiaitons ranging from X-rays to radio. Let's explore the GRBs also detected in X-rays. As shown above, we can obtain HiPs for the XMM-Newton Telescope. XMM-Newton is a powerful telescope that observes the X-ray Universe.

We will repeat the process we used to find Fermi HiPS, but for XMM-Newton:

In [13]:
xmm_hips = MOCServer.find_datasets(meta_data="obs_description=*xmm*&&hips_*=*", casesensitive=False)
xmm_hips

hips_service_url_2,hips_tile_width,obs_copyright,obs_description,bib_reference_url,dataproduct_type,dataproduct_subtype,hips_rgb_red,hips_sampling,hips_progenitor_url,hips_version,hips_initial_dec,obs_copyright_url,hips_rgb_green,hips_status_2,hips_status_1,em_min,hips_order_min,obs_ack,client_application,moc_order,em_max,hips_frame,s_pixel_scale,hips_creation_date,data_pixel_bitpix,client_category,hips_check_code,obs_collection,hips_creator,hips_release_date,hips_pixel_scale,hips_initial_ra,prov_progenitor,hips_builder,obs_initial_ra,obs_initial_fov,hips_service_url,bib_reference,ID,hips_pixel_bitpix,hips_pixel_cut,hips_estsize,hips_overlay,moc_time_range,t_min,obs_regime,hips_status,hipsgen_date,hipsgen_params,moc_type,hips_rgb_blue,hips_tile_format,hips_hierarchy,obs_initial_dec,t_max,moc_sky_fraction,obs_title,TIMESTAMP,creator_did,moc_time_order,hips_data_range,hips_order,hips_nb_tiles,hips_initial_fov,hips_copyright,hips_service_url_1
str58,float64,str24,str153,str64,str5,str5,str30,object,str43,float64,float64,str27,str32,str24,str24,float64,float64,str42,str10,float64,float64,str10,float64,str17,object,str15,str30,str14,str38,str17,float64,float64,str55,str22,float64,float64,str33,str19,str21,object,str13,float64,object,float64,float64,str5,str26,str17,str214,str5,str31,str8,str6,float64,float64,float64,str62,float64,str27,float64,str18,float64,float64,float64,str30,str55
https://alaskybis.cds.unistra.fr/SSC/xcatdb_P_XMM_PN_color,512.0,(c) ESA / SSC XMM-Newton,Colored image made with all PN images of the 4xmmdr12: Red=0.5-1Kev Green=1-2Kev Blue=2-4.5Kev - Healpixilized by Michel L. [Observatoire de Strasbourg],https://ui.adsabs.harvard.edu/abs/2020A%26A...641A.136W/abstract,image,color,red [1.0E-6 NaN 4.0E-4 Linear],--,--,1.4,30.65994167,https://xmmssc.irap.omp.eu/,green [1.0E-6 NaN 4.0E-4 Linear],public mirror unclonable,public mirror unclonable,2.7552e-10,0.0,"Galhecos-Team (Strasbourg), SSC XMM-Newton",AladinLite,7.0,6.1992e-10,equatorial,--,2023-06-05T14:24Z,--,Image/X-ray/XMM,--,XMM PN colored,L. Michel [Observatoire de Strasbourg],2023-06-05T14:31Z,0.0008946,23.4621,XMM-Newton 4XMM PN images and exposure maps,Aladin/HipsGen v12.060,23.4621,0.4580648549089874,https://xcatdb.unistra.fr/PNColor,2020A&A...641A.136W,xcatdb/P/XMM/PN/color,--,--,--,overlayMean mergeOverwriteTile treeMedian,1.0,51577.0,X-ray,public master clonableOnce,2023-06-05T14:24Z,inRed=/bigarchesbox/michel/xmm_hips/hips_red inGreen=/bigarchesbox/michel/xmm_hips/hips_green/ inBlue=/bigarchesbox/michel/xmm_hips/hips_blue out=/bigarchesbox/michel/xmm_hips/RGB RGB id=ivo://xcatdb/P/XMM/PN/color,stmoc,test [1.0E-6 NaN 4.0E-4 Linear],png,--,30.65994167,59912.0,0.09202,False color X-ray images (Red=0.5-1 Green=1-2 Blue=2-4.5 Kev ),1698394027940.0,ivo://xcatdb/P/XMM/PN/color,25.0,--,7.0,--,1.0,(c) Observatoire de Strasbourg,https://alasky.cds.unistra.fr/SSC/xcatdb_P_XMM_PN_color
https://alaskybis.cds.unistra.fr/SSC/xcatdb_P_XMM_PN_eb2,512.0,(c) ESA / SSC XMM-Newton,Image made with all PN images of the 4xmmdr13 in band 0.5-1Kev - Healpixilized by L. Michel [Observatoire de Strasbourg],https://ui.adsabs.harvard.edu/abs/2020A%26A...641A.136W/abstract,image,--,--,"['bilinear', 'bilinear']",https://xcatdb.unistra.fr/PNRed/HpxFinder,1.4,30.65994167,https://xmmssc.irap.omp.eu/,--,public mirror unclonable,public mirror unclonable,2.7552e-10,0.0,"Galhecos-Team (Strasbourg), SSC XMM-Newton",--,7.0,6.1992e-10,equatorial,0.001111,2023-06-05T13:26Z,"['-64', '-64']",Image/X-ray/XMM,png:2167619521 fits:1120067023,4XMM PN eb2,L. Michel [Observatoire de Strasbourg],2023-06-05T13:27Z,0.0008946,23.4621,"XMM-Newton 4XMM PN images, exposure and background maps",Aladin/HipsGen v12.060,23.4621,0.4580648549089874,https://xcatdb.unistra.fr/PNRed,2020A&A...641A.136W,xcatdb/P/XMM/PN/eb2,"['-64', '-64']",1.0E-6 4.0E-4,73959916.0,"['mean', 'overlayMean mergeOverwriteTile treeMean']",1.0,51577.0,X-ray,public master clonableOnce,2023-06-05T13:26Z,"in=/bigarchesbox/michel/xmm_hips/red ou

In [14]:
xmm_hips["ID"]

xcatdb/P/XMM/PN/color
xcatdb/P/XMM/PN/eb2
xcatdb/P/XMM/PN/eb3
xcatdb/P/XMM/PN/eb4


Let's have a closer look at `CDS/P/Fermi/color`, and `xcatdb/P/XMM/PN/color`

#### Setting surveys in ipyaladin from python

In [15]:
aladin.survey = "CDS/P/Fermi/color"

In [16]:
aladin.coo_frame = "galactic"

In [17]:
aladin.overlay_survey="xcatdb/P/XMM/PN/color"

In [18]:
aladin.overlay_survey_opacity = 0.8

That's great, we now have superposed two surveys in `ipyaladin`.
HiPS surveys are built out of mosaic images. We see from the XMM example that they do not necessarilly cover the whole sky, but can also be constructed for partial coverages.

We will not build a HiPS catalog now, but if one day you'd like to build and publish your own, you can look at [this very nice HiPS building tutorial](https://aladin.cds.unistra.fr/tutorials/OV-France-2024/)

Let's now see how we can retrieve GRB events that would have been observed by these telescopes.

## Interacting with databases programmatically

There are two main ways to interact with astronomical databases from python.
- with `pyVO`, the python module that offers Virtual Observatory protocols,
- with `astroquery` modules. If there is one that exists for your database, then it is often really nice to use. But **it is less general than pyVO that works for every VO database**. We used this second approach for the MOCServer, `astroquery.mocserver` was an astroquery module specially written for the MOCServer.

Here, we will focus on the `pyVO` approach, to stick to VO protocols. But most [CDS's python tutorials](https://github.com/cds-astro/tutorials/notebooks) will illustrate the modules `astroquery.vizier`, `astroquery.xmatch`and `astroquery.simbad`, if you are interrested.

### Using the Table Access Protocol (TAP) with pyVO

TAP is a VO standard that allows to interogate tables that have relations with each other with the Astronomical Data Query Language (ADQL).

A database that offers TAP access always have:
- an url that allows to instantiate a TAP service
- a description of its content that is always in the same tables with standardized names. Here we will use **tables**, and **columns**.

#### SIMBAD

![tap_simbad](tap_simbad.png)
Figure 3: (a) Finding the SIMBAD TAP URL, (b) The tables `basic` and `otypedef` have a common column: `otype`, (c) Once a TAP result is added to `ipyaladin`, one can click on the symbols or select subsets to inspect the results further.

Let's try to find GRBs with SIMBAD's TAP access.
First, we need SIMBAD's TAP URL. It can be found on [SIMBAD's webpage](http://simbad.cds.unistra.fr/simbad/), see Figure 3a.

In [20]:
tap_simbad = pyvo.dal.TAPService("https://simbad.cds.unistra.fr/simbad/sim-tap/")

The queries written in ADQL should be written in a string and called with the `search` method. Let's look at the tables available through TAP. `*` means that we want to get everything, and `FROM tables` restricts is to the table named `tables`.

In [21]:
tap_simbad.search("SELECT * FROM tables").to_table()

table_index,schema_name,table_name,table_type,description,utype
int32,object,object,object,object,object
-1,public,basic,table,General data about an astronomical object,
-1,public,ids,view,all names concatenated with pipe,
-1,public,alltypes,view,all object types concatenated with pipe,
-1,public,ident,table,Identifiers of an astronomical object,
-1,public,cat,table,Catalogues name,
-1,public,flux,table,Magnitude/Flux information about an astronomical object,
-1,public,allfluxes,view,"all flux/magnitudes U,B,V,I,J,H,K,u_,g_,r_,i_,z_",
-1,public,filter,table,Description of a flux filter,
-1,public,has_ref,table,Associations between astronomical objects and their bibliographic references,


This ADQL query (`SELECT * FROM tables`) will work in every TAP service, since `tables` is one of the mandatory tables.

Here, we find that SIMBAD is actually made of 35 tables. Some of them have a name that start by `TAP_SCHEMA`. These are the mandatory tables that describe the TAP service.

An other query that will always work is to count the number of columns in all tables of a TAP service, since `columns` is also one of the mandatory tables.

In [22]:
tap_simbad.search("SELECT COUNT(*) FROM columns").to_table()

COUNT_ALL
int64
326


However, the tables that don't start with `TAP_SCHEMA` are specific to each database.

To find which tables of SIMBAD offers a description of the object types, let's add a criteria to our first query:

In [23]:
tap_simbad.search("SELECT * FROM tables WHERE description LIKE '%type%'").to_table()

table_index,schema_name,table_name,table_type,description,utype
int32,object,object,object,object,object
-1,public,alltypes,view,all object types concatenated with pipe,
-1,public,mesVar,table,Collection of stellar variability types and periods.,
-1,public,otypedef,table,all names and definitions for the object types,
-1,public,otypes,table,List of all object types associated with an object,
-1,public,mesSpT,table,Collection of spectral types.,


In ADQL, `%` means 'any character'. We learn that the definition of the object types in SIMBAD is in `otypedef`. Let's look at the first 5 records of the table to see what it looks like.

In [24]:
tap_simbad.search("SELECT TOP 5 * FROM otypedef").to_table()

otype,label,description,comment,is_candidate,otype_shortname,otype_longname,path
object,object,object,object,int16,object,object,object
XB?,,X-ray Binary Candidate,,0,XB?,X-ray Binary Candidate,
Pl?,,Extra-solar Planet Candidate,,0,Pl?,Extra-solar Planet Candidate,
RNe,RefNeb,Reflection Nebula,,0,RNe,Reflection Nebula,ISM > Cld > GNe > RNe
LP?,,Long-Period Variable Candidate,,0,LP?,Long-Period Variable Candidate,
HB?,,Horizontal Branch Star Candidate,,0,HB?,Horizontal Branch Star Candidate,


We can now build an ADQL query to find the code `otype` (the first column) for Gamma Ray Bursts. Since only the columns `otype`, `description`, and `path` look insterresting to us, let's do a query without `*` for the first time:

In [25]:
tap_simbad.search("SELECT otype, description, path FROM otypedef WHERE description LIKE '%Gamma%'").to_table()

otype,description,path
object,object,object
gB,Gamma-ray Burst,gam > gB
gam,Gamma-ray Source,gam


We got it. The code is `gB`. 

This information was extracted from one of SIMBAD's tables. We will use an other one in the next query. If you're interested in learning more about SIMBAD's structure, and how to build more complex ADQL queries for it, you can have a look at [SIMBAD's ADQL documentation](https://cds.unistra.fr/help/documentation/simbad-more/adql-simbad/), or the [documentation of the `astroquery.simbad` module](https://astroquery.readthedocs.io/en/latest/simbad/simbad.html#query-tap) that has a `query_tap` direct access.

Let's now do a request on the `basic` table that contains all the basic information about the objects. `basic` has a lot on columns. You can inspect them with an ADQL query (`SELECT * FROM columns WHERE table_name = 'basic'`) but we already selected the coordinates (`ra`, `dec`), the object type (`otype`) and the number of articles that mention the GRB (`nbref`):

In [26]:
grbs_simbad = tap_simbad.search("""SELECT main_id, ra, dec, otype, nbref FROM basic
WHERE otype = 'gB'""").to_table()
grbs_simbad

main_id,ra,dec,otype,nbref
,deg,deg,,
object,float64,float64,object,int32
Fermi bn170510217,160.15391666666665,-39.21250000000001,gB,19
GRB 060923B,238.19475,-30.90313888888888,gB,38
Fermi bn121011469,260.2136249999999,41.11022222222222,gB,51
GRB 060513,--,--,gB,2
GRB 220103A,--,--,gB,2
GRB 220104A,--,--,gB,4
GRB 220108A,--,--,gB,1
GRB 220107A,169.807,34.17063888888889,gB,21


There are currently 12449 GRBs in SIMBAD. There could be more when you'll execute the notebook. Some of them have position estimates, some don't. This is because SIMBAD is also a bibliographical database, it does not only map objects onto the sky, it also maps objects mentionned in papers. 

For example here, let's look at the GRB with the most citations (`nbref`)

In [27]:
grbs_simbad.sort("nbref", reverse=True)
grbs_simbad

main_id,ra,dec,otype,nbref
,deg,deg,,
object,float64,float64,object,int32
GRB 990123,231.37083333333334,44.75,gB,993
GRB 970508,103.455,79.27194444444444,gB,870
GBS 1900+14,286.804,9.326,gB,865
GRB 970228,75.44458333333334,11.781388888888888,gB,638
GRB 060614,320.88391666666666,-53.02669444444444,gB,626
GRB 021004,6.727833333333334,18.92822222222222,gB,550
GRB 090510,333.5525,-26.597527777777778,gB,503
GRB 990510,204.5318333333333,-80.49688888888889,gB,489


It looks like the "most famous" GRB is `GRB 990123`.

# Pooja can you add fun stuff about it here?

Let's add the ones with an estimated position to the `ipyaladin` widget.

In [28]:
aladin.add_table(grbs_simbad, color="pink", shape="cross")

We immediately see that some fall within the XMM observations. This is nice, but GRBs have a short lifetime span and SIMBAD only has space information, not time. 

We will need to go to more specific data sources. To do so, let's find other TAP services that could have GRBs-specific information.

#### But where do we find TAP URLs??

For this, we will use one extra-powerful tool: the Virtual Observatory registry. This is where any VO service is... registered! It is accessible through [web access](http://dc.g-vo.org/wirr/q/ui/fixed). It is also what TOPCat uses behind the scenes when you're looking for a resource. Here, we will access it thanks to `PyVO`.

In [29]:
# we first define text, servicetype and wavebands constraints
text_constraint = pyvo.registry.Freetext("Gamma", "Ray", "Burst")
service_type_constrains = pyvo.registry.Servicetype("tap").include_auxiliary_services()
waveband_constraint = pyvo.registry.Waveband("radio")

tap_services_GRBs_radio = pyvo.registry.search(text_constraint, service_type_constrains, waveband_constraint)
tap_services_GRBs_radio

<DALResultsTable length=26>
                ivoid                      res_type     ... cap_descriptions
                                                        ...                 
                object                      object      ...      object     
------------------------------------- ----------------- ... ----------------
       ivo://cds.vizier/j/a+a/562/a70 vs:catalogservice ...                 
      ivo://cds.vizier/j/a+a/618/a104 vs:catalogservice ...                 
      ivo://cds.vizier/j/a+a/627/a106 vs:catalogservice ...                 
      ivo://cds.vizier/j/a+a/671/a116 vs:catalogservice ...                 
       ivo://cds.vizier/j/a+a/675/a99 vs:catalogservice ...                 
      ivo://cds.vizier/j/a+a/684/a211 vs:catalogservice ...                 
       ivo://cds.vizier/j/apj/746/156 vs:catalogservice ...                 
        ivo://cds.vizier/j/apj/806/52 vs:catalogservice ...                 
         ivo://cds.vizier/j/apj/814/1 vs:catalog

In [42]:
tap_services_GRBs_radio.to_table()[["res_title", "ivoid"]]

res_title,ivoid
object,object
Imaging GRB 980425 in millimetric and submm,ivo://cds.vizier/j/a+a/562/a70
"NGC3278, SN2009bb host, ATCA and MUSE data",ivo://cds.vizier/j/a+a/618/a104
HI observations of AT 2018cow,ivo://cds.vizier/j/a+a/627/a106
GRB 210731A flux measurements,ivo://cds.vizier/j/a+a/671/a116
FRBs search with Fermi-LAT,ivo://cds.vizier/j/a+a/675/a99
Gaussian model fit components for 3C345,ivo://cds.vizier/j/a+a/684/a211
Radio afterglow observations of GRBs,ivo://cds.vizier/j/apj/746/156
8 Fermi GRB afterglows follow-up,ivo://cds.vizier/j/apj/806/52
"GRB 120326A, 100418A & 100901A multi-wavelength obs.",ivo://cds.vizier/j/apj/814/1


We find 26 TAP services that mention gamma ray burst in the radio waveband! Most, but not all of them, are VizieR catalogs. Let's inspect the first and last ones in more detail.

In [57]:
tap_services_GRBs_radio[0].describe(verbose=True)

Imaging GRB 980425 in millimetric and submm
Short Name: J/A+A/562/A70
IVOA Identifier: ivo://cds.vizier/j/a+a/562/a70
Access modes: tap#aux
- tap#aux: http://tapvizier.cds.unistra.fr/TAPVizieR/tap

Gamma-ray bursts (GRBs) have been proposed as a tool to study star formation
in the Universe, so it is crucial to investigate whether their host galaxies
and immediate environments are in any way special compared with other star-
forming galaxies. Here we present spatially resolved maps of dust emission of
the host galaxy of the closest known GRB 980425 at z=0.0085 using our new
high-resolution observations from Herschel, APEX, ALMA and ATCA. We modeled
the spectral energy distributions of the host and of the star-forming region
displaying the Wolf-Rayet signatures in the spectrum (WR region), located
800pc away from the GRB position. The host is characterised by low dust
content and high fraction of UV-visible star-formation, similar to other dwarf
galaxies. Such galaxies are abundant in th

In [32]:
tap_services_GRBs_radio[-1].describe()

Radio-Selected Gamma-Ray Burst Afterglow Catalog
Short Name: RSSGRBAG
IVOA Identifier: ivo://nasa.heasarc/rssgrbag
Access modes: tap#aux
- tap#aux: https://heasarc.gsfc.nasa.gov/xamin/vo/tap

This table contains a catalog of radio afterglow observations of gamma-ray
bursts (GRBs) over a 14 year period from 1997 to 2011. This sample of 304
afterglows consists of 2,995 flux density measurements (including upper
limits) at frequencies between 0.6 GHz and 660 GHz, with the majority of data
taken in the 8.5-GHz frequency band (1,539 measurements). The authors use this
dataset to carry out a statistical analysis of the radio-selected sample. The
detection rate of radio afterglows stayed unchanged almost at 31% before and
after the launch of the Swift satellite. The canonical long-duration GRB radio
light curve at 8.5 GHz peaks at three to six days in the source rest frame,
with a median peak luminosity of 10<sup>31</sup> erg/s/Hz. The peak radio
luminosities for short-hard bursts, X-ray flas

While doing your own research, you'll surely read all of them with care. Here, let's look at the `Radio-Selected Gamma-Ray Burst Afterglow Catalog`, that we got from the previous research. We read from the description that it is built from a VizieR catalog (`J/ApJ/746/156`).

We can use the HEASARC TAP service  and find its URL like this:

In [33]:
tap_heasarc = tap_services_GRBs_radio["ivo://nasa.heasarc/rssgrbag"].get_service("tap")
tap_heasarc.baseurl

'https://heasarc.gsfc.nasa.gov/xamin/vo/tap'

And we can use the exact same ADQL syntax than with SIMBAD! Let's look at the first 10 tables from the generic `tables` table.

In [34]:
tap_heasarc.search("SELECT TOP 10 table_name from TAP_SCHEMA.tables").to_table()

table_name
object
xmmcfrscat
a2point
pccs857ghz
hmxbcat2
pccs070ghz
tycho2
vvds20cm
sdsss82xmm
a2pic


It's usually good prectice to check the size of a table before dowloading it.

In [43]:
len_rssgrbag = tap_heasarc.search("SELECT COUNT(*) from rssgrbag")
len_rssgrbag

<DALResultsTable length=1>
count
int64
-----
  304

304 lines is quite reasonable, let's download the `Radio-Selected Gamma-Ray Burst Afterglow Catalog`.

In [49]:
rssgrbag = tap_heasarc.search("select * from rssgrbag").to_table()
rssgrbag

<DALResultsTable length=304>
__row      name    source_flag ...      __y_ra_dec          __z_ra_dec    
                               ...                                        
object    object      object   ...       float64             float64      
------ ----------- ----------- ... -------------------- ------------------
     1  GRB 020410             ...   0.0946563709825235 -0.994197106376368
     2  GRB 020321             ...  -0.0501100617750167 -0.993895502347812
     3 GRB 071112B           o ...  -0.0269315139517957  -0.98736960218701
     4  GRB 990510             ...   -0.150203770418467  -0.98627599779725
     5  GRB 990627             ...    0.199074692583951 -0.974672364694757
     6  GRB 011121           n ...    -0.23994849422267 -0.970415240404613
     7  GRB 990712           n ...    0.264710532798964 -0.958361347222492
     8  GRB 990705             ...    0.066531114121275 -0.951762643160288
     9  GRB 970402             ...   -0.260103644584088 -0.935649515818

Let's inspect the result. We find that the is no time information about when the burst was observed. What a pain! Let's still add it to the `ipyaladin` view to compare it to SIMBAD's result and we'll make a more precise query to the VO registry in the next part.

In [44]:
aladin.add_table(rssgrbag, color="blue", shape="circle", source_size=17)

#### The UCD standard

We can actually add a constraint on the content of the columns in a registry search. This uses an other VO standard: the Unified Content Descriptor (UCD). You can get the UCD corresponding to a word on the [UCD Builder page](https://cds.unistra.fr//UCD/cgi-bin/descr2ucd). Here if we enter `time` we get the suggestion to use `time.epoch` as an UCD. Let's try that in the registry.

We already had defined the constraints `waveband_constraint`, `text_constraint`, and `service_type_constraint`

In [48]:
ucd_constraint = pyvo.registry.UCD("time.epoch")

grbs_with_time_tap_services = pyvo.registry.search(ucd_constraint, 
                                              text_constraint,
                                              service_type_constrains,
                                              waveband_constraint
                                             )
grbs_with_time_tap_services

<DALResultsTable length=16>
               ivoid                 ...
                                     ...
               object                ...
------------------------------------ ...
     ivo://cds.vizier/j/a+a/671/a116 ...
      ivo://cds.vizier/j/a+a/675/a99 ...
     ivo://cds.vizier/j/a+a/684/a211 ...
      ivo://cds.vizier/j/apj/746/156 ...
       ivo://cds.vizier/j/apj/806/52 ...
        ivo://cds.vizier/j/apj/814/1 ...
      ivo://cds.vizier/j/apj/859/134 ...
      ivo://cds.vizier/j/apj/884/121 ...
      ivo://cds.vizier/j/apj/895/l23 ...
      ivo://cds.vizier/j/apj/902/l43 ...
       ivo://cds.vizier/j/apj/925/15 ...
      ivo://cds.vizier/j/apjs/218/11 ...
      ivo://cds.vizier/j/apjs/232/17 ...
   ivo://cds.vizier/j/mnras/473/1512 ...
ivo://cds.vizier/j/other/nat/463.513 ...
         ivo://nasa.heasarc/rssgrbag ...

This reduces the number of results. However, we notice that `rssgrbag` is still there. This is an example of a metadata mistake. If you inspect the table we got from the TAP query, you'll see that the durations columns have the UCD `time.epoch` instead of `time.duration` like they should. 

The origine VizieR table from which `rssgrbag` was built, `J/ApJ/746/156`, does not appear in the list. Let's inspect the UCDs of the original table. To do so, let's use the VizieR TAP interface. 

The link can once again either be found on VizieR pages, or through PyVO.

In [51]:
tap_vizier = pyvo.dal.TAPService("https://tapvizier.cds.unistra.fr/TAPVizieR/tap")

VizieR is **huge**. Let's count its tables with the generic table `tables`

In [ ]:
tap_vizier.search("SELECT COUNT(*) FROM tables").to_table()

There are currently 57893 tables in VizieR. Wait wait... This is way bigger than the number of cataloges that is printed under `key numbers` in [CDS's webpages](https://cds.unistra.fr/).

This is because VizieR counts `catalogs` that correspond to published articles, while the `tables` that we just counted thanks to TAP are *comprised* in these catalogs. To sum it up, a catalog has one or more tables.

To help navigate its tables, VizieR has more meta data tables than what is mandatory in TAP services. These additional descriptive tables are called `metaviz` and can be retrieved with:

In [52]:
# access to vizier metadata
result = tap_vizier.search("select * from TAP_SCHEMA.tables where schema_name = 'metaviz'").to_table()
result.pprint_all() # pprint does not hide lines when the result is long

schema_name   table_name   table_type                                                  description                                                   utype nrows
----------- -------------- ---------- -------------------------------------------------------------------------------------------------------------- ----- -----
    metaviz        METAmor      table                                Contains details of Anchor Generation ( Francois Ochsenbein, Isabelle Bourekeb)          31
    metaviz        METApop      table                           Popularity of catalog including all clones ( Francois Ochsenbein, Isabelle Bourekeb)           0
    metaviz        METAucd      table                                     UCD1+ Unified Content Descriptor ( Francois Ochsenbein, Isabelle Bourekeb)         515
    metaviz       METAtime       view                                                  Time catalogue view ( Francois Ochsenbein, Isabelle Bourekeb)           0
    metaviz        METAdba      ta

You might have noticed when we explored SIMBAD that it had two `schema_name`, one was the mandatory `TAP_SCHEMA`, and the other was `public` that contains every public tables of SIMBAD.

`metaviz` is one of VizieR's schemas. We find here that `METAcat` contains information about the catalogs. Let's look for the vizier catalogs that we got from the registry search.

In [92]:
list_catalogs = grbs_with_time_tap_services.to_table()["ivoid"].data.data
list_catalogs

array(['ivo://cds.vizier/j/a+a/671/a116',
       'ivo://cds.vizier/j/a+a/675/a99',
       'ivo://cds.vizier/j/a+a/684/a211',
       'ivo://cds.vizier/j/apj/746/156', 'ivo://cds.vizier/j/apj/806/52',
       'ivo://cds.vizier/j/apj/814/1', 'ivo://cds.vizier/j/apj/859/134',
       'ivo://cds.vizier/j/apj/884/121', 'ivo://cds.vizier/j/apj/895/l23',
       'ivo://cds.vizier/j/apj/902/l43', 'ivo://cds.vizier/j/apj/925/15',
       'ivo://cds.vizier/j/apjs/218/11', 'ivo://cds.vizier/j/apjs/232/17',
       'ivo://cds.vizier/j/mnras/473/1512',
       'ivo://cds.vizier/j/other/nat/463.513',
       'ivo://nasa.heasarc/rssgrbag'], dtype=object)

In [93]:
# let's remove the last one
list_catalogs = list_catalogs[:-1]
list_catalogs

array(['ivo://cds.vizier/j/a+a/671/a116',
       'ivo://cds.vizier/j/a+a/675/a99',
       'ivo://cds.vizier/j/a+a/684/a211',
       'ivo://cds.vizier/j/apj/746/156', 'ivo://cds.vizier/j/apj/806/52',
       'ivo://cds.vizier/j/apj/814/1', 'ivo://cds.vizier/j/apj/859/134',
       'ivo://cds.vizier/j/apj/884/121', 'ivo://cds.vizier/j/apj/895/l23',
       'ivo://cds.vizier/j/apj/902/l43', 'ivo://cds.vizier/j/apj/925/15',
       'ivo://cds.vizier/j/apjs/218/11', 'ivo://cds.vizier/j/apjs/232/17',
       'ivo://cds.vizier/j/mnras/473/1512',
       'ivo://cds.vizier/j/other/nat/463.513'], dtype=object)

In [94]:
# and remove the beginning of the strings to keep only the catalog's names
list_catalogs = [catalog.replace("ivo://cds.vizier/", "") for catalog in list_catalogs]
list_catalogs

['j/a+a/671/a116',
 'j/a+a/675/a99',
 'j/a+a/684/a211',
 'j/apj/746/156',
 'j/apj/806/52',
 'j/apj/814/1',
 'j/apj/859/134',
 'j/apj/884/121',
 'j/apj/895/l23',
 'j/apj/902/l43',
 'j/apj/925/15',
 'j/apjs/218/11',
 'j/apjs/232/17',
 'j/mnras/473/1512',
 'j/other/nat/463.513']

Sometimes, to inspect what the output of a TAP query will be without downloading data for nothing, it's practical  to ask for zero rows in a table:

In [95]:
test = tap_vizier.search("select top 0 * from METAcat")

In [96]:
test.fieldnames

('catid',
 'kslot',
 'status',
 'catype',
 'name',
 'title',
 'bibcode',
 'authors',
 'explain',
 'popu',
 'authid',
 'flags',
 'doi',
 'orcid',
 'artdoi')

Let's write a criteria on the catalog's name (every TAP endpoint can define additional methods. Here we use `cds_lower` to put the catalog names in lower case. Even if they are all different, the additional TAP methods can all be discovered by visiting their URL + capabilities, for SIMBAD it is http://simbad.cds.unistra.fr/simbad/sim-tap/capabilities, for HEASARC https://heasarc.gsfc.nasa.gov/xamin/vo/tap/capabilities, and for VizieR http://tapvizier.cds.unistra.fr/TAPVizieR/tap/capabilities):

In [100]:
catalogs = tap_vizier.search(f"select * from METAcat WHERE cds_lower(name) IN {tuple(list_catalogs)}").to_table()
catalogs[["catid", "title", "bibcode"]]

catid,title,bibcode
int32,object,object
36710116,"GRB 210731A flux measurements (de Wet+, 2023)",2023A&A...671A.116D
36750099,"FRBs search with Fermi-LAT (Principe+, 2023)",2023A&A...675A..99P
36840211,"Gaussian model fit components for 3C345 (Roder+, 2024)",2024A&A...684A.211R
19250015,"Radio afterglow emission of GRBs (Levine+, 2022)",2022ApJ...925...15L
1038046301,"Radio observations of SN 2009bb (Soderberg+, 2010)",2010Natur.463..513S
18140001,"GRB 120326A, 100418A & 100901A multi-wavelength obs. (Laskar+, 2015)",2015ApJ...814....1L
19029043,"Fermi/GBM 2019 and 2020 bursts of SGR J1935+2154 (Lin+, 2020)",2020ApJ...902L..43L
18590134,"A VLA study of high-redshift GRBs. II. GRB140304A (Laskar+, 2018)",2018ApJ...859..134L
17460156,"Radio afterglow observations of GRBs (Chandra+, 2012)",2012ApJ...746..156C


The `catid` is really important here. It is a column that can be used to join other tables to this one. 

When we were navigating SIMBAD, we noticed that the tables `otypedef` and `basic` both had a column with the otype code (remember, it was `gB` for gamma ray bursts). This is what makes a database `relational`. Because there are relations between the tables, see **Figure 3b**.

We actually want to see information about the columns of all tables in all these catalogs. This means we will need to join `METAcat`, that has the catalogs, `METAtab` that describes the tables `METAcol` with the columns information, and `METAucd` with the UDC information. You can find the descriptions of these tables when we looked for the `metaviz` schema.

In [112]:
time_columns = tap_vizier.search(f"""
SELECT METAcol.name AS col_name, 
METAcol.explain as col_description,
METAucd.name as UCD, 
METAcat.name as catalog,
METAcat.catid
FROM METAcat
JOIN METAcol ON METAcat.catid = METAcol.catid
JOIN METAucd ON METAucd.ucdid = METAcol.ucdid
WHERE cds_lower(METAcat.name) IN {tuple(list_catalogs)}
AND METAucd.name = 'time.epoch' 
""").to_table()
time_columns

col_name,col_description,UCD,catalog,catid
object,object,object,object,int32
tm,"[1.3,179.1] Peak time (1)",time.epoch,J/ApJ/746/156,17460156
tm/(1+z),"[0.6,97.6]? Rest-frame peak time (1)",time.epoch,J/ApJ/746/156,17460156
T90,[4.5/115] The T_90_ time,time.epoch,J/ApJ/806/52,18060052
Obs,UTC date of midpoint of observation,time.epoch,J/ApJ/806/52,18060052
delt,[0.01/34] Time of relative to GBM trigger,time.epoch,J/ApJ/806/52,18060052
JD,UT date converted in JD by CDS,time.epoch,J/ApJ/806/52,18060052
delt,[0.1/48] Time of relative to GBM trigger,time.epoch,J/ApJ/806/52,18060052
t,[0.001/19.5] Time since discovery; t-t_0_,time.epoch,J/ApJ/814/1,18140001
t,[0.0002/21.4] Time since discovery; t-t_0_,time.epoch,J/ApJ/814/1,18140001


Based on the columns descriptions that we just gathered and the titles of the catalogs, let's explore `J/A+A/675/A99`.

In [114]:
tables_in_catalog = tap_vizier.search("SELECT name from METAtab where catid = '36750099'").to_table()
tables_in_catalog

name
object
J/A+A/675/A99/tablec1


There is only one table, let's see how many events it contains before downloading it.

In [126]:
len_frbs = tap_vizier.search('SELECT count(*) from "J/A+A/675/A99/tablec1"').to_table()
len_frbs

COUNT_ALL
int64
1020


In [127]:
frbs = tap_vizier.search('SELECT * from "J/A+A/675/A99/tablec1"').to_table()
frbs

recno,Name,RAJ2000,DEJ2000,DM,z,Dist,Fradio,Flradio,Time,Width,Lradio,Enradio,TS10s,Fg10s,EFg10s,Lg10s,Eng10s,TS100s,Fg100s,EFg100s,Lg100s,Eng100s,TS1000s,Fg1000s,EFg1000s,Lg1000s,Eng1000s,TS10000s,Fg10000s,EFg10000s,Lg10000s,Eng10000s,DbestTrip,Tripptpval,TripDur
,,deg,deg,pc.cm**-3,,Mpc,Jy,Jy ms,s,s,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,s,,s
int32,object,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
758,FRB20190131D,57.5,22.8,642.1,0.469726,2708.562,3.0,7.0,602219234,0.00017,2.6333562391649693e+43,4.1807109559682987e+40,-1.0341711982608803e-07,0.00010975716267874176,0.026388,4.5893295293497535e+49,4.589329529349754e+50,-1.0142943052487974e-05,1.0815691452957217e-05,0.003002,4.9195810651424715e+48,4.919581065142472e+50,-6.609190094764017e-06,1.409258168495037e-06,0.00073,1.0311695821447441e+48,1.031169582144744e+51,-4.930996055918513e-08,4.7078008246779593e-07,0.00025926603938069664,3.625158741974752e+47,3.625158741974752e+51,826.298899,1.0,17025.288263
618,FRB20181117C,53.2,25.7,1776.0,1.379612,10087.5,1.6,3.0,595756081,0.0001,1.94804090203769e+44,1.5349463134385325e+41,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,17.678494,0.999999999999996,11045.425679
735,FRB20190118B,39.7,23.6,666.4,0.512272,3004.123,0.3,4.0,601094851,0.004,3.239423545026393e+42,2.8561205318394294e+40,--,--,--,--,--,--,--,--,--,--,-5.278000190539478e-06,2.4782279197515816e-05,0.010836,1.955733990976748e+49,1.955733990976748e+52,-9.439195082450169e-06,1.483037836509699e-07,0.0012302069883572467,1.7532215605590992e+48,1.7532215605590993e+52,53.464842,1.0,67262.164522
699,FRB20181231A,29.7,21.0,1376.5,1.080872,7467.797,1.1,2.0,599541491,0.00071,7.339874495784818e+43,6.413285662152385e+40,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0.5881564447246888,5.342919306136272e-07,0.00019927200826170101,2.582253956393315e+48,2.582253956393315e+52,937.256625,1111111111.0,234867.857473
133,FRB20190213,33.5,20.06667,651.45,0.506921,2966.585,0.5,3.0,611260418,--,5.2649540646077556e+42,2.0963090944538839e+40,-8.340957782314307e-06,6.366242864009937e-05,0.018145,3.581696182927081e+49,3.581696182927081e+50,-3.138779253575308e-05,6.738213230427825e-06,0.001525,3.34919321536876e+48,3.3491932153687602e+50,-2.077655516075083e-05,7.479805920917404e-07,0.000243,4.598717758650443e+47,4.598717758650443e+50,-1.3876439197701984e-07,6.419414755523065e-07,0.00013579853147351146,3.1141809074424412e+47,3.1141809074424414e+51,--,1.0,29161.537896
134,FRB20190213,33.5,20.06667,651.45,0.506921,2966.585,--,0.6,603247337,--,--,4.1926181889077686e+39,--,--,--,--,--,--,--,--,--,--,-2.0495278592116972e-06,5.0889521520984825e-05,0.0167,3.1494960376881924e+49,3.1494960376881926e+52,-1.3230891227067332e-06,2.7953676045874645e-07,0.0001259713020322539,2.201474663089865e+47,2.201474663089865e+51,--,1.0,45496.165084
646,FRB20181203A,33.6,23.6,635.7,0.491955,2862.147,1.7,4.0,597129899,0.00017,1.6662641145002562e+43,2.627841789785418e+40,--,--,--,--,--,--,--,--,--,--,-5.819912195809129e-06,1.2390140557331069e-05,0.002906,5.778449461382513e+48,5.778449461382513e+51,-1.540697667223867e-07,6.080932584367833e-07,0.00018648135674685822,3.326313319654859e+47,3.326313319654859e+51,1137.013575,1.0,51635.269212
754,FRB20190129A,45.1,21.4,478.8,0.360961,1985.241,0.5,5.0,602043938,0.0011,2.3578037071477543e+42,1.7324550756207782e+40,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,-8.25877464194491e-08,4.264042627732611e-07,0.00021544678113915425,1.6165798653189955e+47,1.6165798653189955e+51,915.108361,1.0,61868.518122


In [128]:
# In VizieR, the time information is stored in seconds since the first of january 2000
from astropy.time.formats import erfa, TimeFromEpoch
class TimeVizier(TimeFromEpoch):
    name = "viztime"
    unit = 1.0 / erfa.DAYSEC  # in days (1 day == 86400 seconds)
    epoch_val = "2000-01-01 00:00:00"
    epoch_val2 = None
    epoch_scale = "utc"
    epoch_format = "iso"

In [129]:
frbs["astropy_time"] = Time(frbs["Time"], scale='utc', format="viztime").iso
frbs

recno,Name,RAJ2000,DEJ2000,DM,z,Dist,Fradio,Flradio,Time,Width,Lradio,Enradio,TS10s,Fg10s,EFg10s,Lg10s,Eng10s,TS100s,Fg100s,EFg100s,Lg100s,Eng100s,TS1000s,Fg1000s,EFg1000s,Lg1000s,Eng1000s,TS10000s,Fg10000s,EFg10000s,Lg10000s,Eng10000s,DbestTrip,Tripptpval,TripDur,astropy_time
,,deg,deg,pc.cm**-3,,Mpc,Jy,Jy ms,s,s,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,,cm**-2.s**-1,mW.m**-2,1e-07W,1e-07J,s,,s,
int32,object,float64,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str23
758,FRB20190131D,57.5,22.8,642.1,0.469726,2708.562,3.0,7.0,602219234,0.00017,2.6333562391649693e+43,4.1807109559682987e+40,-1.0341711982608803e-07,0.00010975716267874176,0.026388,4.5893295293497535e+49,4.589329529349754e+50,-1.0142943052487974e-05,1.0815691452957217e-05,0.003002,4.9195810651424715e+48,4.919581065142472e+50,-6.609190094764017e-06,1.409258168495037e-06,0.00073,1.0311695821447441e+48,1.031169582144744e+51,-4.930996055918513e-08,4.7078008246779593e-07,0.00025926603938069664,3.625158741974752e+47,3.625158741974752e+51,826.298899,1.0,17025.288263,2019-01-31 03:07:14.000
618,FRB20181117C,53.2,25.7,1776.0,1.379612,10087.5,1.6,3.0,595756081,0.0001,1.94804090203769e+44,1.5349463134385325e+41,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,17.678494,0.999999999999996,11045.425679,2018-11-17 07:48:01.000
735,FRB20190118B,39.7,23.6,666.4,0.512272,3004.123,0.3,4.0,601094851,0.004,3.239423545026393e+42,2.8561205318394294e+40,--,--,--,--,--,--,--,--,--,--,-5.278000190539478e-06,2.4782279197515816e-05,0.010836,1.955733990976748e+49,1.955733990976748e+52,-9.439195082450169e-06,1.483037836509699e-07,0.0012302069883572467,1.7532215605590992e+48,1.7532215605590993e+52,53.464842,1.0,67262.164522,2019-01-18 02:47:31.000
699,FRB20181231A,29.7,21.0,1376.5,1.080872,7467.797,1.1,2.0,599541491,0.00071,7.339874495784818e+43,6.413285662152385e+40,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,0.5881564447246888,5.342919306136272e-07,0.00019927200826170101,2.582253956393315e+48,2.582253956393315e+52,937.256625,1111111111.0,234867.857473,2018-12-31 03:18:11.000
133,FRB20190213,33.5,20.06667,651.45,0.506921,2966.585,0.5,3.0,611260418,--,5.2649540646077556e+42,2.0963090944538839e+40,-8.340957782314307e-06,6.366242864009937e-05,0.018145,3.581696182927081e+49,3.581696182927081e+50,-3.138779253575308e-05,6.738213230427825e-06,0.001525,3.34919321536876e+48,3.3491932153687602e+50,-2.077655516075083e-05,7.479805920917404e-07,0.000243,4.598717758650443e+47,4.598717758650443e+50,-1.3876439197701984e-07,6.419414755523065e-07,0.00013579853147351146,3.1141809074424412e+47,3.1141809074424414e+51,--,1.0,29161.537896,2019-05-15 18:33:38.000
134,FRB20190213,33.5,20.06667,651.45,0.506921,2966.585,--,0.6,603247337,--,--,4.1926181889077686e+39,--,--,--,--,--,--,--,--,--,--,-2.0495278592116972e-06,5.0889521520984825e-05,0.0167,3.1494960376881924e+49,3.1494960376881926e+52,-1.3230891227067332e-06,2.7953676045874645e-07,0.0001259713020322539,2.201474663089865e+47,2.201474663089865e+51,--,1.0,45496.165084,2019-02-12 00:42:17.000
646,FRB20181203A,33.6,23.6,635.7,0.491955,2862.147,1.7,4.0,597129899,0.00017,1.6662641145002562e+43,2.627841789785418e+40,--,--,--,--,--,--,--,--,--,--,-5.819912195809129e-06,1.2390140557331069e-05,0.002906,5.778449461382513e+48,5.778449461382513e+51,-1.540697667223867e-07,6.080932584367833e-07,0.00018648135674685822,3.326313319654859e+47,3.326313319654859e+51,1137.013575,1.0,51635.269212,2018-12-03 05:24:59.000
754,FRB20190129A,45.1,21.4,478.8,0.360961,1985.241,0.5,5.0,602043938,0.0011,2.3578037071477543e+42,1.7324550756207782e+40,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,-8.25877464194491e-08,4.264042627732611e-07,0.00021544678113915425,1.6165798653189955e+47,1.616579

We can also add this to `ipyaladin`

In [130]:
aladin.add_table(frbs, color="teal", shape="circle", marker_size=10)

To know wether some of these events have been observed at the time the XMM telescope was observing, let's use a Space-Time MOC (STMOC).

## Space-Time MOCs

# Explain MOCs and STMOCs here

 Let's see how we can filter them automatically.

To do so, we will use VizieR's TAP service.

#### VizieR's TAP service

The URL can be found on [VizieR's pages](https://vizier.unistra.fr/) under `Other related services` in the middle of the page. 

In [ ]:
Here, we can read that one of the tables describes 

In [ ]:
metacatcols = tap_vizier.search("select * from columns where table_name = 'METAcat'").to_table()
metacatcols

In [ ]:
To do 

Using this service, we can easily access the log of observations XMM-NEWTON & Fermi.  This allows us to obtain important information that can find specific observations that match certain criteria, such as time of observation, celestial coordinates, target name, or observation ID. Such observation logs often contain information about the data quality, data availability, cross-matching catalogues and much more.

In [ ]:
# catalogs are in metacat
xmm_DR13 = tap_vizier.search("""select top 1 * from METAcat
                  where title like '%XMM%' and title like'%DR13%'
                  """).to_table()
xmm_DR13

In [ ]:
#Vizier.find_catalogs(["xmm", "log"])

In [ ]:
# tables are in metatab
xmm_DR13_tables = tap_vizier.search(
    """select * from METAtab
    where catid = 9069
    """
).to_table()
xmm_DR13_tables

In [ ]:
# retrieve the whole xmmlog table
xmmlog = tap_vizier.search('select * from "IX/69/summary"').to_table()
xmmlog


In [ ]:
len(xmmlog)

In [ ]:
for column in xmmlog.colnames:
    print(column, ":", xmmlog[column].description)

In [ ]:
aladin.add_table(xmmlog, color="lightpink")

# I get an error - TypeError: add_table() got an unexpected keyword argument 'color'

Now chose a source and use the simbad tool

## Now build the STMOC

# 
STMOCs are useful for identifying temporal and spatial overlaps between different astronomical catalogs, which is particularly useful for studying GRBs. Since GRBs are transient events that can be observed across multiple wavelengths.

Using STMOCs can:
- Help in identifying overlaps between different observations from different instruments which can aid in confirming GRB detections.
- Help in obtaing information from different instruments and to perform various physical studies such as GRB light curves, spectra, and afterglows, leading to better modeling of the underlying physical processes.
- Help in pinpointing the exact sky regions where GRBs were observed, aiding in follow-up observations with other telescopes.
- Help in cross-matching process that can lead to statiscal studies of GRB populations.


#
To proceed we can use tap service to understand the structure and content of time-related data in the catalog.

In [ ]:
# information on time is in metatime
time_columns = tap_vizier.search("select * from columns where table_name='METAtime'")
time_columns

In [ ]:
# lets get metatime for the second table (catid=2) of the catalog (catid=9096)
xmm_DR13_tables = tap_vizier.search(
    """select * from METAtime JOIN METAcol
    on METAtime.catid = METAcol.catid
    and METAcol.colid = METAtime.colid
    and METAcol.tabid = METAtime.tabid
    where METAcol.catid = 9069 and METAcol.tabid = 2
    """
).to_table()
xmm_DR13_tables

# 
STMOCs can handle only specific format of time. Hence, we convert Modified Julian Date (MJD) columns to Astropy Time objects for easier manipulation and consistency.

In [ ]:
# TODO here check the time scale in the catalog ('tai', 'tcb', 'tcg', 'tdb', 'tt', 'ut1', 'utc', 'local')
# got help from Laurent https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/uhb/reftime.html
xmmlog["start_obs"] = Time(xmmlog["MJD0"], format='mjd', scale="tt")
xmmlog["end_obs"] = Time(xmmlog["MJD1"], format='mjd', scale="tt")
xmmlog

# 
To create STMOCs, it is very important to set the temporal and spatial resolution. 
We have chosen 0.5 days to account for daily varations in the GRB (???)
We have chosen 1 degree as its adequate resoltion for identifying a GRB (???)

In [ ]:
TimeMOC.time_resolution_to_order(TimeDelta(0.5, format="jd")) # 0.5 days is order 26

In [ ]:
MOC.spatial_resolution_to_order(Angle("1d")) # 1 degrees is order 6

In [ ]:
stmoc_xmm = STMOC.from_time_ranges_positions(
    times_start=xmmlog["start_obs"],
    times_end=xmmlog["end_obs"],
    lon = Longitude(xmmlog["RAJ2000"]),
    lat = Latitude(xmmlog["DEJ2000"]),
    time_depth=26, spatial_depth=6
)

# 
Shown below are a series of plots that visualize the STMOCs maps of XMM-Newton observations for different time ranges. The plots show how the patial coverage of XMM-Newton observations changes over time. This is important for understanding the temporal distribution of GRB observations. Such plots can be useful in identifying overlapping observation periods with other instruments.

In [ ]:
def add_to_plot(fig, label, wcs, title, moc):
    """Add a MOC to a plot."""
    ax = fig.add_subplot(label, projection=wcs)

    ax.grid(color="black", linestyle="dotted")
    ax.set_title(title)
    ax.set_xlabel("lon")
    ax.set_ylabel("lat")

    moc.fill(ax=ax, wcs=wcs, alpha=0.9, fill=True, linewidth=0, color="#00bb00")
    # moc.border(ax=ax, wcs=wcs, linewidth=1, color="green")


fig = plt.figure(figsize=(10, 10))

time_ranges = Time(
    [
        [["2000-01-01", "2005-01-01"]],
        [["2005-01-01", "2010-01-01"]],
        [["2010-01-01", "2015-01-01"]],
        [["2015-01-01", "2020-01-01"]],
    ],
    format="iso",
    scale="tdb",
    out_subfmt="date",
)
with WCS(
    fig,
    fov=330 * u.deg,
    center=SkyCoord(0, 0, unit="deg", frame="galactic"),
    coordsys="galactic",
    rotation=Angle(0, u.degree),
    projection="AIT",
) as wcs:
    for i in range(0, 4):
        tmoc = TimeMOC.from_time_ranges(
            min_times=time_ranges[i][0, 0],
            max_times=time_ranges[i][0, 1],
            delta_t=TimeDelta(0.5, scale="tdb", format="jd"),
        )

        moc_2mass = stmoc_xmm.query_by_time(tmoc)
        title = "XMM observations between \n{} and {}".format(
            time_ranges[i][0, 0].iso,
            time_ranges[i][0, 1].iso,
        )
        id_subplot = int("22" + str(i + 1))
        add_to_plot(fig, id_subplot, wcs, title, moc_2mass)

plt.show()

## Now fermi grb

#
The same process can be repeated for Fermi 

In [ ]:
fermi_grb_metacat = tap_vizier.search("""select * from METAcat
                              where title like '%Fermi%GRB%'
                              """).to_table()
fermi_grb_metacat


In [ ]:
fermi_grb_catid = 18930046

In [ ]:
fermi_grb_tables = tap_vizier.search(f"""select * from METAtab where catid={fermi_grb_catid}
""").to_table()
fermi_grb_tables

In [ ]:
fermi_grb_triggers_columns = tap_vizier.search("""select * from columns where table_name='J/ApJ/893/46/table4'""").to_table()
fermi_grb_triggers_columns

In [ ]:
fermi_grb_durations_columns = tap_vizier.search("""select * from columns where table_name='J/ApJ/893/46/table5'""").to_table()
fermi_grb_durations_columns

In [ ]:
# lets get metatime for the trigger table (tabid=1)
fermi_triggers_time_info = tap_vizier.search(
    f"""select * from METAtime JOIN METAcol
    on METAtime.catid = METAcol.catid
    and METAcol.colid = METAtime.colid
    and METAcol.tabid = METAtime.tabid
    where METAcol.catid = {fermi_grb_catid} and METAcol.tabid = 1
    """
).to_table()
fermi_triggers_time_info


In [ ]:
# let's query the joined tables (to get event start time and duration)
fermi = tap_vizier.search(
    """select * from "J/ApJ/893/46/table5"
    join "J/ApJ/893/46/table4" USING(Fermi)
    """
).to_table()
fermi

In [ ]:
# we have to define a specific time for vizier 

from astropy.time.formats import erfa, TimeFromEpoch
class TimeVizier(TimeFromEpoch):
    name = "viztime"
    unit = 1.0 / erfa.DAYSEC  # in days (1 day == 86400 seconds)
    epoch_val = "2000-01-01 00:00:00"
    epoch_val2 = None
    epoch_scale = "utc"
    epoch_format = "iso"

In [ ]:
fermi["start_time"] = Time(fermi["Obs"] - erfa.DAYSEC / 2., format="viztime", scale="utc")
fermi["end_time"] = Time(fermi["Obs"] + erfa.DAYSEC / 2., format="viztime", scale="utc")
fermi

In [ ]:
stmoc_fermi = STMOC.from_time_ranges_positions(
    times_start=fermi["start_time"],
    times_end=fermi["end_time"],
    lon = Longitude(fermi["RAJ2000"]),
    lat = Latitude(fermi["DEJ2000"]),
    time_depth=26, spatial_depth=5  # ask Ada why 5 here
)

# 
Now using the STMOC information for both XMM and Fermi, we can use intersection method to identify the common space-time regions covered by both Fermi and XMM-Newton observations. Additionally, all the events, that have been identified, are confirmed by two different telescopes, therefore we can know for certain that the GRBs identified are genuine. These give as output the time when both the telescopes with pointing at the same area of the sky. These come in handy specially for identifing simultaneous observations, which are crucial for multi-wavelength studies of transient events like Gamma-Ray Bursts (GRBs).

In [ ]:
intersection = stmoc_fermi.intersection(stmoc_xmm)
intersection

# 
By now we have seen the power of using STMOCs to identifying transcients events such as GRBs. This method can also be applied for 
different astrophysical objects such as supernova remnants which emit at different wavelengths. STMOCs is a powerful tool which
facilitates the identification of temporal and spatial overlaps, enabling comprehensive multi-wavelength analyses and efficient data management, ultimately advancing our understanding of the object
of interest.

In [ ]:
hess = "J/A+A/612/A3"
super_nova_remnant = "RX J1713.7-3946"

In [ ]:
from astroquery.simbad import Simbad

In [ ]:
Simbad.query_object("RX J1713.7-3946")

In [ ]:
help(intersection.contains)

In [ ]:
only_space = MOC.from_string("6/37910 16282 9576 17030 26013")

In [ ]:
from astropy.coordinates import Longitude, Latitude

In [ ]:
only_space.contains_lonlat(lon=Longitude("17 12 27", unit="deg"), lat=Latitude("-39 41.2", unit="deg"))

In [ ]:
only_space.query_vizier_table("J/A+A/612/A3")

In [ ]:
fermi_grbs_intersection = only_space.query_vizier_table("J/ApJ/893/46")
fermi_grbs_intersection

In [ ]:
aladin.add_table(fermi_grbs_intersection, color="hotpink", shape="cross")

In [ ]:
simbad = Simbad()
simbad.add_votable_fields("flux")

In [ ]:
simbad_result = simbad.query_objects([f"Fermi {id}" for id in fermi_grbs_intersection["Fermi"]])
simbad_result

In [ ]:
only_space.query_vizier_table("IX/69")